In [1]:
pip install boto3


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Administrator\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [16]:
import boto3

bucket = "pdfsunobucket"

In [14]:

#to check bucket contents in cli : aws s3 ls s3://textract-console-us-east-1-c23eb1d7-6fdb-498f-9b1d-06eacbf7fc11/


def upload_file_to_s3(file_name, object_name=None):
    """
    Upload a file to an S3 bucket

    :param file_name: File to upload
    :param object_name: S3 object name. If not specified, file_name is used
    :return: True if file was uploaded, else False
    """
    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        s3_client.upload_file(file_name, bucket, object_name)
    except Exception as e:
        print(e)
        return False
    return True

# Example usage
upload_file_to_s3('C:/Users/Administrator/Desktop/PDF Suno/Bridging_the_Divide.pdf' , 'trial_pdf_1')


True

In [23]:
bucket_name = 'textract-console-us-east-1-c23eb1d7-6fdb-498f-9b1d-06eacbf7fc11'  # Replace with your actual bucket name

def start_document_text_detection(document):
    # Ensure the Textract client is initialized in the correct region
    textract = boto3.client('textract', region_name='us-east-1')  # Replace 'us-east-1' with your bucket's region
    response = textract.start_document_text_detection(
        DocumentLocation={'S3Object': {'Bucket': bucket_name, 'Name': document}}
    )
    print ("response:", response)
    return response['JobId']

def is_job_complete(jobId):
    """
    Check if the Textract job is complete and handle different statuses.
    """
    print("is_job_complete ID:", jobId)
    textract = boto3.client('textract', region_name='us-east-1')
    try:
        response = textract.get_document_text_detection(JobId=jobId)
        status = response["JobStatus"]
        print("Current job status:", status)
        
        if status == "SUCCEEDED":
            return True, response
        elif status == "FAILED":
            print("Job failed.")
            return False, response
        else:
            # For "IN_PROGRESS" and any other statuses
            return False, response
    except Exception as e:
        print(f"Error checking job status: {e}")
        # Decide how you want to handle exceptions. You might want to retry or fail the operation.
        return False, None

def get_job_results(jobId):
    """
    Get the results of a completed Textract job
    """
    print("get job results ID: ", jobId)
    pages = []
    textract = boto3.client('textract', region_name='us-east-1')
    response = textract.get_document_text_detection(JobId=jobId)
    pages.extend(response["Blocks"])
    
    while "NextToken" in response:
        response = textract.get_document_text_detection(JobId=jobId, NextToken=response["NextToken"])
        pages.extend(response["Blocks"])
    
    return pages

# Start the Textract job
jobId = start_document_text_detection('trial_pdf_1')
print("function call id:",jobId)

# Wait for the job to complete (you may want to implement a more robust waiting mechanism)
while True:
    jobComplete, response = is_job_complete(jobId)
    if jobComplete:
        print("Job completed successfully.")
        # Process the successful response here
        break
    elif response is None or response["JobStatus"] == "FAILED":
        # Handle failure or error in job status check
        break
    else:
        print("Job is still in progress. Waiting before checking again...")
        time.sleep(10)  # Adjust sleep time as necessary based on expected job completion time

# Get the job results
results = get_job_results(jobId)

# Extract text from the results
text = "\n".join([block["Text"] for block in results if block["BlockType"] == "LINE"])
print(text)


response: {'JobId': 'a97773ef03a67691cd6e896c517386deea8132f0e5b0496aa53de215b1c8127b', 'ResponseMetadata': {'RequestId': '56acb132-5cbc-4f2d-89c1-741b8c1573d3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '56acb132-5cbc-4f2d-89c1-741b8c1573d3', 'content-type': 'application/x-amz-json-1.1', 'content-length': '76', 'date': 'Thu, 22 Feb 2024 16:24:19 GMT'}, 'RetryAttempts': 0}}
function call id: a97773ef03a67691cd6e896c517386deea8132f0e5b0496aa53de215b1c8127b
is_job_complete ID: a97773ef03a67691cd6e896c517386deea8132f0e5b0496aa53de215b1c8127b
Current job status: IN_PROGRESS
Job is still in progress. Waiting before checking again...
is_job_complete ID: a97773ef03a67691cd6e896c517386deea8132f0e5b0496aa53de215b1c8127b
Current job status: IN_PROGRESS
Job is still in progress. Waiting before checking again...
is_job_complete ID: a97773ef03a67691cd6e896c517386deea8132f0e5b0496aa53de215b1c8127b
Current job status: IN_PROGRESS
Job is still in progress. Waiting before checking agai

In [24]:
# Initialize the Amazon Translate client
translate_client = boto3.client(service_name='translate', region_name='us-east-1', use_ssl=True)

# Your extracted text in English
english_text = text
# Translate the text from English to Hindi
result = translate_client.translate_text(Text=english_text, 
                                         SourceLanguageCode="en", 
                                         TargetLanguageCode="hi")

# The translated text in Hindi
hindi_text = result.get('TranslatedText')

print(hindi_text)

विभाजन को दूर करना: सभी कैसे बंद हो सकते हैं
लैंग्वेज गैप्स और फोस्टर ग्लोबल
अंडरस्टैंडिंग
भाषा की बाधाओं ने लंबे समय तक सभी संस्कृतियों में संचार, समझ और सहयोग में बाधा डाली है।
व्यापार वार्ताओं से लेकर वैज्ञानिक आदान-प्रदान तक, विचारों और विचारों का निर्बाध रूप से अनुवाद करने में असमर्थता
इससे गलतफहमी हो सकती है, अवसर छूट सकते हैं और यहां तक कि संघर्ष भी हो सकता है। सौभाग्य से, का उद्भव
आर्टिफिशियल इंटेलिजेंस (AI) आशा की किरण पेश करता है, जो इन कमियों को दूर करने और इसे और अधिक बढ़ावा देने का वादा करता है
कनेक्टेड वर्ल्ड।
भाषा अनुवाद के लिए AI का उपयोग करने के सबसे महत्वपूर्ण लाभों में से एक इसकी सटीकता में निहित है और
दक्षता। पारंपरिक मशीनी अनुवाद तकनीकें अक्सर बारीकियों, मुहावरों और सांस्कृतिक चीजों से जूझती हैं
संदर्भ, जिसके कारण रुके हुए और गलत अनुवाद होते हैं। हालाँकि, AL-संचालित अनुवाद प्रणालियाँ
उन्नत न्यूरल नेटवर्क का उपयोग करें, जो बड़ी मात्रा में टेक्स्ट डेटा से सीख सकते हैं, जिससे वे
भाषा की सूक्ष्म जटिलताओं को कैप्चर करें और अधिक स्वाभाविक और सटीक अनुवाद तैयार करें। यह
सं

In [ ]:
polly_client = boto3.client(service_name='polly', region_name='us-east-1', use_ssl=True)

response = polly_client.synthesize_speech(VoiceId='Aditi',
                                          OutputFormat='mp3',
                                          Text=hindi_text,
                                          LanguageCode='hi-IN')

# Save the audio stream returned by Amazon Polly to a file
with open('speech.mp3', 'wb') as file:
    file.write(response['AudioStream'].read())
